<a href="https://colab.research.google.com/github/JunHyukJang426/Prompt_Tuning/blob/main/prompt_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
pip install torch

In [ ]:
import json
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
from datasets import load_dataset
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

train_data = [
    {"input": "펀드가 뭐야?", "target": "펀드가 무엇인지 설명해주세요. 또한 펀드의 특징과 종류에 대해 설명해주세요."},
    {"input": "펀드 투자와 주식 투자의 차이점은 무엇인가요?", "target": "펀드 투자와 주식 투자 간의 차이점을 예시와 함께 설명해 주실 수 있나요?"},
    {"input": "펀드 투자의 장단점은 무엇인가요?", "target": "펀드 투자의 장단점을 실제 사례를 들어 설명해 주세요."},
    {"input": "펀드 투자를 시작하려면 어떻게 해야 하나요?", "target": "펀드 투자를 처음 시작하려고 하는데, 필요한 절차와 준비 사항을 알려주세요."},
    {"input": "펀드의 종류에는 어떤 것들이 있나요?", "target": "펀드의 종류에는 어떤 것들이 있고, 각각의 특징과 투자 목적은 무엇인가요?"},
    {"input": "적립식 펀드와 일시불 펀드의 차이점은 무엇인가요?", "target": "적립식 펀드와 일시불 펀드의 차이점과 각 방식의 장단점은 무엇인가요?"},
    {"input": "펀드의 수익률은 어떻게 결정되나요?", "target": "펀드의 수익률은 어떤 요인에 의해 결정되는지 구체적으로 설명해 주세요."},
    {"input": "펀드의 위험 요소는 무엇인가요?", "target": "펀드 투자 시 고려해야 할 주요 위험 요소는 무엇이며, 이를 어떻게 관리할 수 있나요?"},
    {"input": "펀드 수수료는 어떻게 계산되나요?", "target": "펀드 수수료는 어떤 종류가 있고, 각각의 수수료가 투자 수익에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 운용 방식은 어떻게 되나요?", "target": "펀드의 운용 방식에 대해 설명해 주세요. 액티브 펀드와 패시브 펀드의 차이점도 알고 싶어요."},
    {"input": "펀드의 수익을 실현하려면 어떻게 해야 하나요?", "target": "펀드에서 발생한 수익을 실현하는 방법과 그 절차를 상세히 알려주세요."},
    {"input": "펀드를 언제 매수하고 언제 매도해야 하나요?", "target": "펀드를 언제 매수하고 언제 매도하는 것이 좋은지, 시기를 결정하는 기준은 무엇인가요?"},
    {"input": "펀드 투자 시 고려해야 할 사항은 무엇인가요?", "target": "펀드 투자 시 어떤 사항들을 중점적으로 고려해야 하는지 구체적으로 알려주세요."},
    {"input": "특정 펀드를 선택할 때 주의해야 할 점은 무엇인가요?", "target": "특정 펀드를 선택할 때 주의해야 할 점과 그 이유를 예시와 함께 설명해 주세요."},
    {"input": "펀드의 과거 성과는 어떻게 확인할 수 있나요?", "target": "펀드의 과거 성과를 어떻게 확인하고, 이를 통해 미래 성과를 예측하는 방법이 있나요?"},
    {"input": "펀드의 운용 보고서는 어떻게 이해해야 하나요?", "target": "펀드의 운용 보고서를 이해하는 방법과 중요하게 봐야 할 항목들을 알려주세요."},
    {"input": "펀드의 투자 대상은 무엇인가요?", "target": "펀드가 주로 투자하는 대상(주식, 채권, 부동산 등)은 무엇인지 알고 싶어요."},
    {"input": "펀드 투자 시 세금은 어떻게 되나요?", "target": "펀드 투자 시 발생하는 세금 종류와 그 계산 방법을 구체적으로 설명해 주세요."},
    {"input": "펀드의 환매 절차는 어떻게 되나요?", "target": "펀드의 환매 절차와 환매 시 주의해야 할 사항을 자세히 알려주세요."},
    {"input": "펀드 투자의 리스크 관리 방법은 무엇인가요?", "target": "펀드 투자의 리스크를 관리하는 방법과 이를 위해 활용할 수 있는 도구나 전략이 있다면 알려주세요."},
    {"input": "펀드 투자의 기본 개념을 알려주세요.", "target": "펀드가 무엇인지 설명해주세요. 또한 펀드의 특징과 종류에 대해 설명해주세요."},
    {"input": "펀드와 ETF의 차이점이 궁금해요.", "target": "펀드 투자와 ETF 투자 간의 차이점을 예시와 함께 설명해 주세요."},
    {"input": "펀드의 위험은 어떻게 줄일 수 있나요?", "target": "펀드 투자 시 고려해야 할 주요 위험 요소와 이를 관리할 수 있는 방법을 설명해 주세요."},
    {"input": "펀드 투자로 얻는 수익은 어떻게 계산되나요?", "target": "펀드의 수익률은 어떤 요인에 의해 결정되는지 구체적으로 설명해 주세요."},
    {"input": "펀드를 처음 투자하는 사람에게 조언을 주세요.", "target": "펀드 투자를 처음 시작하려고 하는데, 필요한 절차와 준비 사항을 알려주세요."},
    {"input": "적립식 펀드의 장단점은 무엇인가요?", "target": "적립식 펀드와 일시불 펀드의 차이점과 각 방식의 장단점을 설명해 주세요."},
    {"input": "펀드에서 주로 투자하는 자산은 무엇인가요?", "target": "펀드가 주로 투자하는 대상(주식, 채권, 부동산 등)에 대해 설명해 주세요."},
    {"input": "펀드 매매 시 고려할 점은 무엇인가요?", "target": "펀드를 언제 매수하고 언제 매도하는 것이 좋은지, 시기를 결정하는 기준을 설명해 주세요."},
    {"input": "펀드 수수료의 종류와 그 영향은?", "target": "펀드 수수료는 어떤 종류가 있고, 각각의 수수료가 투자 수익에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 투자 목표는 무엇인가요?", "target": "펀드의 종류에는 어떤 것들이 있고, 각각의 특징과 투자 목적은 무엇인가요?"},
    {"input": "펀드의 과거 성과는 어떻게 분석하나요?", "target": "펀드의 과거 성과를 어떻게 확인하고, 이를 통해 미래 성과를 예측하는 방법이 있나요?"},
    {"input": "펀드 투자 시 어떤 전략을 사용해야 하나요?", "target": "펀드 투자의 리스크를 관리하는 방법과 이를 위해 활용할 수 있는 도구나 전략을 설명해 주세요."},
    {"input": "펀드 운용 보고서의 주요 항목은?", "target": "펀드의 운용 보고서를 이해하는 방법과 중요하게 봐야 할 항목들을 설명해 주세요."},
    {"input": "펀드에서 발생한 수익은 어떻게 인출하나요?", "target": "펀드에서 발생한 수익을 실현하는 방법과 그 절차를 상세히 알려주세요."},
    {"input": "펀드 투자 시 발생하는 세금은?", "target": "펀드 투자 시 발생하는 세금 종류와 그 계산 방법을 구체적으로 설명해 주세요."},
    {"input": "펀드의 환매 시 주의할 점은?", "target": "펀드의 환매 절차와 환매 시 주의해야 할 사항을 자세히 설명해 주세요."},
    {"input": "펀드의 수익률 결정 요인은?", "target": "펀드의 수익률은 어떤 요인에 의해 결정되는지 구체적으로 설명해 주세요."},
    {"input": "펀드와 주식의 차이는 무엇인가요?", "target": "펀드 투자와 주식 투자 간의 차이점을 예시와 함께 설명해 주세요."},
    {"input": "펀드의 기본 원리는?", "target": "펀드가 무엇인지 설명해주세요. 또한 펀드의 특징과 종류에 대해 설명해주세요."},
    {"input": "펀드 투자의 위험 요소는?", "target": "펀드 투자 시 고려해야 할 주요 위험 요소와 이를 관리할 수 있는 방법을 설명해 주세요."},
    {"input": "펀드 투자 시 유의할 사항은?", "target": "펀드 투자 시 어떤 사항들을 중점적으로 고려해야 하는지 구체적으로 설명해 주세요."},
    {"input": "펀드의 과거 성과는 어떻게 파악하나요?", "target": "펀드의 과거 성과를 어떻게 확인하고, 이를 통해 미래 성과를 예측하는 방법이 있나요?"},
    {"input": "펀드 매매의 적기는?", "target": "펀드를 언제 매수하고 언제 매도하는 것이 좋은지, 시기를 결정하는 기준을 설명해 주세요."},
    {"input": "펀드 수익을 어떻게 확인하나요?", "target": "펀드에서 발생한 수익을 실현하는 방법과 그 절차를 상세히 설명해 주세요."},
    {"input": "펀드의 주요 투자 대상은?", "target": "펀드가 주로 투자하는 대상(주식, 채권, 부동산 등)에 대해 설명해 주세요."},
    {"input": "펀드의 수익률 계산 방법은?", "target": "펀드의 수익률은 어떤 요인에 의해 결정되는지 구체적으로 설명해 주세요."},
    {"input": "펀드의 위험 관리는?", "target": "펀드 투자의 리스크를 관리하는 방법과 이를 위해 활용할 수 있는 도구나 전략을 설명해 주세요."},
    {"input": "펀드와 ETF의 차이는?", "target": "펀드 투자와 ETF 투자 간의 차이점을 예시와 함께 설명해 주세요."},
    {"input": "펀드 투자 시 필요한 절차는?", "target": "펀드 투자를 처음 시작하려고 하는데, 필요한 절차와 준비 사항을 알려주세요."},
    {"input": "펀드의 투자 전략은?", "target": "펀드 투자의 리스크를 관리하는 방법과 이를 위해 활용할 수 있는 도구나 전략을 설명해 주세요."},
    {"input": "펀드와 채권의 차이점은?", "target": "펀드 투자와 채권 투자 간의 차이점을 예시와 함께 설명해 주세요."},
    {"input": "펀드 수익 실현 방법은?", "target": "펀드에서 발생한 수익을 실현하는 방법과 그 절차를 상세히 설명해 주세요."},
    {"input": "펀드의 과거 성과 분석 방법은?", "target": "펀드의 과거 성과를 어떻게 확인하고, 이를 통해 미래 성과를 예측하는 방법이 있나요?"},
    {"input": "펀드의 환매 절차는?", "target": "펀드의 환매 절차와 환매 시 주의해야 할 사항을 자세히 설명해 주세요."},
    {"input": "펀드 수익률의 영향 요인은?", "target": "펀드의 수익률은 어떤 요인에 의해 결정되는지 구체적으로 설명해 주세요."},
    {"input": "펀드 투자 시 고려할 점은?", "target": "펀드 투자 시 어떤 사항들을 중점적으로 고려해야 하는지 구체적으로 설명해 주세요."},
    {"input": "펀드 수익률은 어떻게 계산되나요?", "target": "펀드의 수익률은 어떤 요인에 의해 결정되는지 구체적으로 설명해 주세요."},
    {"input": "펀드의 과거 성과를 파악하는 방법은?", "target": "펀드의 과거 성과를 어떻게 확인하고, 이를 통해 미래 성과를 예측하는 방법이 있나요?"},
    {"input": "펀드의 주요 투자 자산은?", "target": "펀드가 주로 투자하는 대상(주식, 채권, 부동산 등)에 대해 설명해 주세요."},
    {"input": "펀드의 수수료는 어떻게 구성되나요?", "target": "펀드 수수료는 어떤 종류가 있고, 각각의 수수료가 투자 수익에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 기본 개념을 설명해주세요.", "target": "펀드가 무엇인지 설명해주세요. 또한 펀드의 특징과 종류에 대해 설명해주세요."},
    {"input": "펀드의 수익 실현 절차는?", "target": "펀드에서 발생한 수익을 실현하는 방법과 그 절차를 상세히 설명해 주세요."},
    {"input": "펀드와 주식의 주요 차이점은?", "target": "펀드 투자와 주식 투자 간의 차이점을 예시와 함께 설명해 주세요."},
    {"input": "펀드 투자 시 발생하는 세금은 어떻게 계산되나요?", "target": "펀드 투자 시 발생하는 세금 종류와 그 계산 방법을 구체적으로 설명해 주세요."},
    {"input": "펀드의 환매 시 주의 사항은?", "target": "펀드의 환매 절차와 환매 시 주의해야 할 사항을 자세히 설명해 주세요."},
    {"input": "펀드 수익률에 영향을 미치는 요인은?", "target": "펀드의 수익률은 어떤 요인에 의해 결정되는지 구체적으로 설명해 주세요."},
    {"input": "펀드 투자의 리스크 관리는 어떻게 하나요?", "target": "펀드 투자의 리스크를 관리하는 방법과 이를 위해 활용할 수 있는 도구나 전략을 설명해 주세요."},
    {"input": "펀드 투자와 채권 투자 간의 주요 차이점은?", "target": "펀드 투자와 채권 투자 간의 차이점을 예시와 함께 설명해 주세요."},
    {"input": "펀드 수익을 실현하는 방법은?", "target": "펀드에서 발생한 수익을 실현하는 방법과 그 절차를 상세히 설명해 주세요."},
    {"input": "펀드의 과거 성과를 어떻게 분석하나요?", "target": "펀드의 과거 성과를 어떻게 확인하고, 이를 통해 미래 성과를 예측하는 방법이 있나요?"},
    {"input": "펀드의 환매 절차와 주의할 점은?", "target": "펀드의 환매 절차와 환매 시 주의해야 할 사항을 자세히 설명해 주세요."},
    {"input": "펀드 수익률에 영향을 미치는 주요 요인은?", "target": "펀드의 수익률은 어떤 요인에 의해 결정되는지 구체적으로 설명해 주세요."},
    {"input": "펀드 투자 시 유의할 사항은?", "target": "펀드 투자 시 어떤 사항들을 중점적으로 고려해야 하는지 구체적으로 설명해 주세요."},
    {"input": "펀드의 기본 원리는 무엇인가요?", "target": "펀드가 무엇인지 설명해주세요. 또한 펀드의 특징과 종류에 대해 설명해주세요."},
    {"input": "펀드 수익률의 계산 방법은?", "target": "펀드의 수익률은 어떤 요인에 의해 결정되는지 구체적으로 설명해 주세요."},
    {"input": "펀드의 주요 투자 대상은 무엇인가요?", "target": "펀드가 주로 투자하는 대상(주식, 채권, 부동산 등)에 대해 설명해 주세요."},
    {"input": "펀드의 수수료 구조는 어떻게 되나요?", "target": "펀드 수수료는 어떤 종류가 있고, 각각의 수수료가 투자 수익에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 수익 실현 절차를 설명해주세요.", "target": "펀드에서 발생한 수익을 실현하는 방법과 그 절차를 상세히 설명해 주세요."},
    {"input": "펀드 투자와 주식 투자 간의 주요 차이점은?", "target": "펀드 투자와 주식 투자 간의 차이점을 예시와 함께 설명해 주세요."},
    {"input": "펀드 투자 시 발생하는 세금은 어떻게 계산되나요?", "target": "펀드 투자 시 발생하는 세금 종류와 그 계산 방법을 구체적으로 설명해 주세요."},
    {"input": "펀드의 환매 시 주의 사항은?", "target": "펀드의 환매 절차와 환매 시 주의해야 할 사항을 자세히 설명해 주세요."},
    {"input": "펀드 수익률에 영향을 미치는 주요 요인은?", "target": "펀드의 수익률은 어떤 요인에 의해 결정되는지 구체적으로 설명해 주세요."},
    {"input": "펀드 투자의 리스크 관리는 어떻게 하나요?", "target": "펀드 투자의 리스크를 관리하는 방법과 이를 위해 활용할 수 있는 도구나 전략을 설명해 주세요."},
    {"input": "펀드 투자와 채권 투자 간의 차이점은?", "target": "펀드 투자와 채권 투자 간의 차이점을 예시와 함께 설명해 주세요."},
    {"input": "펀드의 수익률에 어떤 요인이 영향을 미치나요?", "target": "펀드의 수익률이 결정되는 주요 요인은 무엇인지 구체적으로 설명해 주세요."},
    {"input": "펀드의 수익 실현 방법을 알려주세요.", "target": "펀드에서 발생한 수익을 실현하는 방법과 그 절차를 자세히 설명해 주세요."},
    {"input": "펀드와 예금의 차이는?", "target": "펀드 투자와 예금의 차이점과 각각의 장단점을 설명해 주세요."},
    {"input": "펀드의 장기 투자 전략은?", "target": "펀드의 장기 투자 전략에 대해 설명해 주세요. 장기 투자의 장점도 포함해 주세요."},
    {"input": "펀드의 단기 투자 전략은?", "target": "펀드의 단기 투자 전략에 대해 설명해 주세요. 단기 투자의 장점과 단점을 포함해 주세요."},
    {"input": "펀드의 유동성은 어떻게 되나요?", "target": "펀드의 유동성에 대해 설명해 주세요. 유동성 높은 펀드와 낮은 펀드의 차이점도 포함해 주세요."},
    {"input": "펀드의 수수료는 어떤 종류가 있나요?", "target": "펀드 수수료의 종류와 각각의 특징, 그리고 투자 수익에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 환매 시 손실이 발생할 수 있나요?", "target": "펀드의 환매 시 손실이 발생할 수 있는 경우와 이를 최소화할 수 있는 방법을 설명해 주세요."},
    {"input": "펀드의 설정일이란?", "target": "펀드의 설정일이 무엇인지 설명해 주세요. 설정일이 펀드에 미치는 영향도 포함해 주세요."},
    {"input": "펀드의 분산 투자란?", "target": "펀드의 분산 투자에 대해 설명해 주세요. 분산 투자의 장점과 단점을 포함해 주세요."},
    {"input": "펀드의 펀드 매니저 역할은?", "target": "펀드 매니저의 역할과 책임에 대해 설명해 주세요. 좋은 펀드 매니저를 선택하는 방법도 포함해 주세요."},
    {"input": "펀드의 투자 지역은?", "target": "펀드가 투자하는 주요 지역과 각 지역의 투자 특징에 대해 설명해 주세요."},
    {"input": "펀드의 투자 시점은?", "target": "펀드 투자의 적절한 시점을 결정하는 방법과 고려해야 할 요소들을 설명해 주세요."},
    {"input": "펀드의 분배금이란?", "target": "펀드의 분배금이 무엇인지, 분배금이 발생하는 이유와 분배금의 종류에 대해 설명해 주세요."},
    {"input": "펀드의 인덱스 투자는?", "target": "인덱스 펀드의 특징과 장단점, 인덱스 투자가 어떻게 이루어지는지 설명해 주세요."},
    {"input": "펀드의 리밸런싱이란?", "target": "펀드의 리밸런싱이 무엇인지, 리밸런싱이 필요한 이유와 방법에 대해 설명해 주세요."},
    {"input": "펀드의 합병은?", "target": "펀드의 합병이 무엇인지, 합병이 투자자에게 미치는 영향에 대해 설명해 주세요."},
    {"input": "펀드의 상환은?", "target": "펀드 상환이 무엇인지, 상환 절차와 상환 시 주의해야 할 점을 설명해 주세요."},
    {"input": "펀드의 가치 평가 방법은?", "target": "펀드의 가치를 평가하는 방법과 이를 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 배당 수익은?", "target": "펀드의 배당 수익이 무엇인지, 배당 수익의 계산 방법과 배당 수익이 발생하는 이유를 설명해 주세요."},
    {"input": "펀드의 지속 가능성은?", "target": "펀드의 지속 가능성이란 무엇인지, 지속 가능한 펀드를 선택하는 방법을 설명해 주세요."},
    {"input": "펀드의 거래 비용은?", "target": "펀드의 거래 비용이 무엇인지, 거래 비용의 종류와 이를 줄이는 방법을 설명해 주세요."},
    {"input": "펀드의 벤치마크는?", "target": "펀드의 벤치마크가 무엇인지, 벤치마크가 펀드에 미치는 영향과 벤치마크를 선택하는 방법을 설명해 주세요."},
    {"input": "펀드의 투자 철학은?", "target": "펀드의 투자 철학이 무엇인지, 투자 철학이 펀드 운용에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 공모와 사모는?", "target": "펀드의 공모와 사모의 차이점, 각 방식의 장단점을 설명해 주세요."},
    {"input": "펀드의 목표 설정은?", "target": "펀드의 목표 설정이란 무엇인지, 펀드 목표 설정의 중요성과 목표 설정 방법을 설명해 주세요."},
    {"input": "펀드의 리스크와 리턴 관계는?", "target": "펀드의 리스크와 리턴의 관계, 이를 통해 투자 결정을 내리는 방법을 설명해 주세요."},
    {"input": "펀드의 투자 금액은?", "target": "펀드 투자 금액을 결정하는 방법과 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 사후 관리는?", "target": "펀드 투자 후 사후 관리의 중요성, 사후 관리 방법과 이를 통해 얻을 수 있는 이점을 설명해 주세요."},
    {"input": "펀드의 고객 서비스는?", "target": "펀드 운용사의 고객 서비스가 무엇인지, 좋은 고객 서비스를 제공하는 펀드를 선택하는 방법을 설명해 주세요."},
    {"input": "펀드의 환매 수수료는?", "target": "펀드의 환매 수수료가 무엇인지, 환매 수수료의 종류와 환매 수수료를 줄이는 방법을 설명해 주세요."},
    {"input": "펀드의 투자 한도는?", "target": "펀드의 투자 한도가 무엇인지, 투자 한도가 펀드에 미치는 영향과 이를 설정하는 방법을 설명해 주세요."},
    {"input": "펀드의 리스크 평가 방법은?", "target": "펀드의 리스크를 평가하는 방법과 이를 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 수익률 비교는?", "target": "펀드의 수익률을 비교하는 방법과 수익률 비교 시 고려해야 할 요소들을 설명해 주세요."},
    {"input": "펀드의 신용 리스크는?", "target": "펀드의 신용 리스크가 무엇인지, 신용 리스크를 평가하고 관리하는 방법을 설명해 주세요."},
    {"input": "펀드의 리스크 분산은?", "target": "펀드의 리스크 분산이란 무엇인지, 리스크 분산을 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 트래킹 에러는?", "target": "펀드의 트래킹 에러가 무엇인지, 트래킹 에러를 평가하고 관리하는 방법을 설명해 주세요."},
    {"input": "펀드의 운용 자산은?", "target": "펀드의 운용 자산이 무엇인지, 운용 자산의 종류와 운용 자산이 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 관리 비용은?", "target": "펀드의 관리 비용이 무엇인지, 관리 비용의 종류와 이를 줄이는 방법을 설명해 주세요."},
    {"input": "펀드의 시장 리스크는?", "target": "펀드의 시장 리스크가 무엇인지, 시장 리스크를 평가하고 관리하는 방법을 설명해 주세요."},
    {"input": "펀드의 투자 스타일은?", "target": "펀드의 투자 스타일이 무엇인지, 각 스타일의 특징과 장단점을 설명해 주세요."},
    {"input": "펀드의 투자 철학은?", "target": "펀드의 투자 철학이 무엇인지, 투자 철학이 펀드 운용에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 펀드 오브 펀드는?", "target": "펀드 오브 펀드의 특징과 장단점, 펀드 오브 펀드가 어떻게 운용되는지 설명해 주세요."},
    {"input": "펀드의 상장폐지 리스크는?", "target": "펀드의 상장폐지 리스크가 무엇인지, 상장폐지 리스크를 평가하고 관리하는 방법을 설명해 주세요."},
    {"input": "펀드의 평가 방법은?", "target": "펀드의 평가 방법이 무엇인지, 펀드를 평가할 때 고려해야 할 요소들을 설명해 주세요."},
    {"input": "펀드의 자산 배분 전략은?", "target": "펀드의 자산 배분 전략이 무엇인지, 자산 배분 전략의 중요성과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 포트폴리오 관리는?", "target": "펀드의 포트폴리오 관리가 무엇인지, 포트폴리오를 관리할 때 고려해야 할 요소들을 설명해 주세요."},
    {"input": "펀드의 글로벌 투자 전략은?", "target": "펀드의 글로벌 투자 전략이 무엇인지, 글로벌 투자 전략의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 기금 조성은?", "target": "펀드의 기금 조성이 무엇인지, 기금 조성의 중요성과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 사회적 책임 투자는?", "target": "펀드의 사회적 책임 투자가 무엇인지, 사회적 책임 투자의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 유동성 리스크는?", "target": "펀드의 유동성 리스크가 무엇인지, 유동성 리스크를 평가하고 관리하는 방법을 설명해 주세요."},
    {"input": "펀드의 투자 관리 시스템은?", "target": "펀드의 투자 관리 시스템이 무엇인지, 투자 관리 시스템의 중요성과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 정기 리포트는?", "target": "펀드의 정기 리포트가 무엇인지, 정기 리포트를 통해 얻을 수 있는 정보와 이를 활용하는 방법을 설명해 주세요."},
    {"input": "펀드의 투자 시 고려 사항은?", "target": "펀드 투자 시 고려해야 할 사항과 이를 통해 성공적인 투자를 할 수 있는 방법을 설명해 주세요."},
    {"input": "펀드의 주식형과 채권형 차이는?", "target": "주식형 펀드와 채권형 펀드의 차이점, 각 펀드의 장단점을 설명해 주세요."},
    {"input": "펀드의 분배 주기는?", "target": "펀드의 분배 주기가 무엇인지, 분배 주기가 투자자에게 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 대체 투자란?", "target": "펀드의 대체 투자에 대해 설명해 주세요. 대체 투자의 장점과 단점을 포함해 주세요."},
    {"input": "펀드의 투자 목표 설정 방법은?", "target": "펀드의 투자 목표를 설정하는 방법과 투자 목표 설정의 중요성을 설명해 주세요."},
    {"input": "펀드의 유동성 관리 방법은?", "target": "펀드의 유동성을 관리하는 방법과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 헤지 전략은?", "target": "펀드의 헤지 전략이 무엇인지, 헤지 전략의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 투자 수익 배분은?", "target": "펀드의 투자 수익 배분이 어떻게 이루어지는지, 수익 배분의 종류와 배분 시 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 세금 효율성은?", "target": "펀드의 세금 효율성이 무엇인지, 세금 효율성을 높이는 방법과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 비과세 혜택은?", "target": "펀드의 비과세 혜택이 무엇인지, 비과세 혜택을 받을 수 있는 조건과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 투자 보고서는?", "target": "펀드의 투자 보고서가 무엇인지, 투자 보고서를 통해 얻을 수 있는 정보와 이를 활용하는 방법을 설명해 주세요."},
    {"input": "펀드의 환율 리스크는?", "target": "펀드의 환율 리스크가 무엇인지, 환율 리스크를 평가하고 관리하는 방법을 설명해 주세요."},
    {"input": "펀드의 시장 전망은?", "target": "펀드의 시장 전망을 어떻게 분석하는지, 시장 전망이 펀드 투자에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 주요 투자 대상은?", "target": "펀드가 주로 투자하는 대상(주식, 채권, 부동산 등)에 대해 설명해 주세요."},
    {"input": "펀드의 인플레이션 헷지는?", "target": "펀드의 인플레이션 헷지가 무엇인지, 인플레이션 헷지를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 주요 산업은?", "target": "펀드가 주로 투자하는 산업과 각 산업의 투자 특징을 설명해 주세요."},
    {"input": "펀드의 리서치 방법은?", "target": "펀드의 리서치 방법이 무엇인지, 리서치를 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 분기 보고서는?", "target": "펀드의 분기 보고서가 무엇인지, 분기 보고서를 통해 얻을 수 있는 정보와 이를 활용하는 방법을 설명해 주세요."},
    {"input": "펀드의 배당 수익률은?", "target": "펀드의 배당 수익률이 무엇인지, 배당 수익률의 계산 방법과 배당 수익률이 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 거래 빈도는?", "target": "펀드의 거래 빈도가 무엇인지, 거래 빈도가 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 매매 타이밍은?", "target": "펀드 매매 시 타이밍을 결정하는 방법과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 사후 평가 방법은?", "target": "펀드의 사후 평가 방법이 무엇인지, 사후 평가를 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 투자 위험 요소는?", "target": "펀드 투자 시 고려해야 할 주요 위험 요소와 이를 관리할 수 있는 방법을 설명해 주세요."},
    {"input": "펀드의 투자 가치 분석은?", "target": "펀드의 투자 가치를 분석하는 방법과 이를 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 환매 기간은?", "target": "펀드의 환매 기간이 무엇인지, 환매 기간이 투자자에게 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 시장 타이밍 전략은?", "target": "펀드의 시장 타이밍 전략이 무엇인지, 시장 타이밍 전략의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 투자 다변화는?", "target": "펀드의 투자 다변화가 무엇인지, 투자 다변화를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 성장 가능성은?", "target": "펀드의 성장 가능성을 평가하는 방법과 이를 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 환매 타이밍은?", "target": "펀드의 환매 타이밍을 결정하는 방법과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 운용 스타일은?", "target": "펀드의 운용 스타일이 무엇인지, 각 스타일의 특징과 장단점을 설명해 주세요."},
    {"input": "펀드의 펀드 리서치는?", "target": "펀드 리서치가 무엇인지, 리서치를 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 수익 분배는?", "target": "펀드의 수익 분배가 어떻게 이루어지는지, 수익 분배의 종류와 분배 시 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 비용 구조는?", "target": "펀드의 비용 구조가 무엇인지, 비용 구조의 종류와 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 투자 분석은?", "target": "펀드의 투자 분석이 무엇인지, 투자 분석을 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 환매 시 고려사항은?", "target": "펀드의 환매 시 고려해야 할 주요 사항과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 비율 투자는?", "target": "펀드의 비율 투자가 무엇인지, 비율 투자의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 수익률 변동성은?", "target": "펀드의 수익률 변동성이 무엇인지, 변동성을 평가하고 관리하는 방법을 설명해 주세요."},
    {"input": "펀드의 자산 규모는?", "target": "펀드의 자산 규모가 무엇인지, 자산 규모가 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 전략적 자산 배분은?", "target": "펀드의 전략적 자산 배분이 무엇인지, 전략적 자산 배분의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 세금 처리 방법은?", "target": "펀드의 세금 처리 방법이 무엇인지, 세금 처리를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 채권 투자는?", "target": "펀드의 채권 투자가 무엇인지, 채권 투자의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 공매도 전략은?", "target": "펀드의 공매도 전략이 무엇인지, 공매도 전략의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 기술적 분석은?", "target": "펀드의 기술적 분석이 무엇인지, 기술적 분석을 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 매매 계획은?", "target": "펀드의 매매 계획이 무엇인지, 매매 계획을 세울 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 성장주 투자는?", "target": "펀드의 성장주 투자가 무엇인지, 성장주 투자의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 가치주 투자는?", "target": "펀드의 가치주 투자가 무엇인지, 가치주 투자의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 고배당 투자는?", "target": "펀드의 고배당 투자가 무엇인지, 고배당 투자의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 인플레이션 리스크는?", "target": "펀드의 인플레이션 리스크가 무엇인지, 인플레이션 리스크를 평가하고 관리하는 방법을 설명해 주세요."},
    {"input": "펀드의 경기 민감도는?", "target": "펀드의 경기 민감도가 무엇인지, 경기 민감도를 평가하고 관리하는 방법을 설명해 주세요."},
    {"input": "펀드의 시장 리서치는?", "target": "펀드의 시장 리서치가 무엇인지, 시장 리서치를 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 자산 배분 원칙은?", "target": "펀드의 자산 배분 원칙이 무엇인지, 자산 배분 원칙의 중요성과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 경기 대응 전략은?", "target": "펀드의 경기 대응 전략이 무엇인지, 경기 대응 전략의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 현금흐름 관리는?", "target": "펀드의 현금흐름 관리가 무엇인지, 현금흐름을 관리할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 수익 실현 전략은?", "target": "펀드의 수익 실현 전략이 무엇인지, 수익 실현 전략의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 포트폴리오 다변화는?", "target": "펀드의 포트폴리오 다변화가 무엇인지, 포트폴리오 다변화의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 자산 선택 기준은?", "target": "펀드의 자산 선택 기준이 무엇인지, 자산 선택 기준을 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 지역별 투자 전략은?", "target": "펀드의 지역별 투자 전략이 무엇인지, 각 지역별 투자 전략의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 환경 리스크는?", "target": "펀드의 환경 리스크가 무엇인지, 환경 리스크를 평가하고 관리하는 방법을 설명해 주세요."},
    {"input": "펀드의 투자 시 참고할 자료는?", "target": "펀드 투자 시 참고할 수 있는 주요 자료와 이를 활용하는 방법을 설명해 주세요."},
    {"input": "펀드의 미래 전망은?", "target": "펀드의 미래 전망을 분석하는 방법과 이를 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 투자 성과는?", "target": "펀드의 투자 성과를 평가하는 방법과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 글로벌 투자 리스크는?", "target": "펀드의 글로벌 투자 리스크가 무엇인지, 글로벌 투자 리스크를 평가하고 관리하는 방법을 설명해 주세요."},
    {"input": "펀드의 가치 투자는?", "target": "펀드의 가치 투자가 무엇인지, 가치 투자의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 미래 성장 가능성은?", "target": "펀드의 미래 성장 가능성을 평가하는 방법과 이를 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 수익 목표는?", "target": "펀드의 수익 목표가 무엇인지, 수익 목표를 설정하는 방법과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 경제적 환경은?", "target": "펀드의 경제적 환경이 무엇인지, 경제적 환경이 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 사회적 환경은?", "target": "펀드의 사회적 환경이 무엇인지, 사회적 환경이 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 기술적 환경은?", "target": "펀드의 기술적 환경이 무엇인지, 기술적 환경이 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 정치적 환경은?", "target": "펀드의 정치적 환경이 무엇인지, 정치적 환경이 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 법적 환경은?", "target": "펀드의 법적 환경이 무엇인지, 법적 환경이 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 인구통계학적 환경은?", "target": "펀드의 인구통계학적 환경이 무엇인지, 인구통계학적 환경이 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 문화적 환경은?", "target": "펀드의 문화적 환경이 무엇인지, 문화적 환경이 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 경쟁 환경은?", "target": "펀드의 경쟁 환경이 무엇인지, 경쟁 환경이 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 산업 환경은?", "target": "펀드의 산업 환경이 무엇인지, 산업 환경이 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 매수 타이밍은?", "target": "펀드의 매수 타이밍을 결정하는 방법과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 매도 타이밍은?", "target": "펀드의 매도 타이밍을 결정하는 방법과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 상장 리스크는?", "target": "펀드의 상장 리스크가 무엇인지, 상장 리스크를 평가하고 관리하는 방법을 설명해 주세요."},
    {"input": "펀드의 상장 전략은?", "target": "펀드의 상장 전략이 무엇인지, 상장 전략의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 자산 관리 방법은?", "target": "펀드의 자산 관리 방법이 무엇인지, 자산 관리 방법을 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 투자 기간은?", "target": "펀드의 투자 기간이 무엇인지, 투자 기간을 설정하는 방법과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 경기 순환 투자는?", "target": "펀드의 경기 순환 투자가 무엇인지, 경기 순환 투자의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 채권 수익률은?", "target": "펀드의 채권 수익률이 무엇인지, 채권 수익률의 계산 방법과 채권 수익률이 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 주식 수익률은?", "target": "펀드의 주식 수익률이 무엇인지, 주식 수익률의 계산 방법과 주식 수익률이 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 부동산 수익률은?", "target": "펀드의 부동산 수익률이 무엇인지, 부동산 수익률의 계산 방법과 부동산 수익률이 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 금 수익률은?", "target": "펀드의 금 수익률이 무엇인지, 금 수익률의 계산 방법과 금 수익률이 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 원자재 수익률은?", "target": "펀드의 원자재 수익률이 무엇인지, 원자재 수익률의 계산 방법과 원자재 수익률이 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 ETF 수익률은?", "target": "펀드의 ETF 수익률이 무엇인지, ETF 수익률의 계산 방법과 ETF 수익률이 펀드에 미치는 영향을 설명해 주세요."},
    {"input": "펀드의 리스크 대비 수익률은?", "target": "펀드의 리스크 대비 수익률이 무엇인지, 리스크 대비 수익률을 평가하는 방법과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 환경, 사회 및 지배구조(ESG) 투자는?", "target": "펀드의 ESG 투자가 무엇인지, ESG 투자의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 장기 수익률은?", "target": "펀드의 장기 수익률이 무엇인지, 장기 수익률을 평가하는 방법과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 단기 수익률은?", "target": "펀드의 단기 수익률이 무엇인지, 단기 수익률을 평가하는 방법과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 투자 철칙은?", "target": "펀드의 투자 철칙이 무엇인지, 투자 철칙을 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 매매 전략은?", "target": "펀드의 매매 전략이 무엇인지, 매매 전략의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 환위험 관리 방법은?", "target": "펀드의 환위험 관리 방법이 무엇인지, 환위험 관리를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 글로벌 투자 기회는?", "target": "펀드의 글로벌 투자 기회가 무엇인지, 글로벌 투자 기회를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 시장 조사 방법은?", "target": "펀드의 시장 조사 방법이 무엇인지, 시장 조사를 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 경기 예측 방법은?", "target": "펀드의 경기 예측 방법이 무엇인지, 경기 예측을 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 투자 타이밍은?", "target": "펀드의 투자 타이밍을 결정하는 방법과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 인플레이션 방어는?", "target": "펀드의 인플레이션 방어가 무엇인지, 인플레이션 방어를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 펀드 매니저 선택 방법은?", "target": "펀드 매니저를 선택하는 방법과 좋은 펀드 매니저를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 환매 수수료 계산 방법은?", "target": "펀드의 환매 수수료를 계산하는 방법과 환매 수수료를 줄이는 방법을 설명해 주세요."},
    {"input": "펀드의 장기 성장성은?", "target": "펀드의 장기 성장성을 평가하는 방법과 이를 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 시장 리스크 관리는?", "target": "펀드의 시장 리스크를 관리하는 방법과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 투자 관리 방법은?", "target": "펀드의 투자 관리 방법이 무엇인지, 투자 관리 방법을 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 환율 변동 리스크는?", "target": "펀드의 환율 변동 리스크가 무엇인지, 환율 변동 리스크를 평가하고 관리하는 방법을 설명해 주세요."},
    {"input": "펀드의 통화 리스크는?", "target": "펀드의 통화 리스크가 무엇인지, 통화 리스크를 평가하고 관리하는 방법을 설명해 주세요."},
    {"input": "펀드의 유동성 위험 관리는?", "target": "펀드의 유동성 위험을 관리하는 방법과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 투자 수익률은?", "target": "펀드의 투자 수익률이 무엇인지, 투자 수익률을 평가하는 방법과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 리스크 관리 전략은?", "target": "펀드의 리스크 관리 전략이 무엇인지, 리스크 관리 전략의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 자산 운용 전략은?", "target": "펀드의 자산 운용 전략이 무엇인지, 자산 운용 전략의 장점과 단점을 설명해 주세요."},
    {"input": "펀드의 시장 분석은?", "target": "펀드의 시장 분석이 무엇인지, 시장 분석을 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 투자 대상 선택 기준은?", "target": "펀드의 투자 대상 선택 기준이 무엇인지, 투자 대상 선택 기준을 통해 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 투자 비중은?", "target": "펀드의 투자 비중이 무엇인지, 투자 비중을 설정하는 방법과 이를 통해 얻을 수 있는 장점을 설명해 주세요."},
    {"input": "펀드의 매매 수수료는?", "target": "펀드의 매매 수수료가 무엇인지, 매매 수수료의 종류와 이를 줄이는 방법을 설명해 주세요."},
    {"input": "펀드의 주식형 펀드의 특징은?", "target": "주식형 펀드의 특징과 장단점, 주식형 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 채권형 펀드의 특징은?", "target": "채권형 펀드의 특징과 장단점, 채권형 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 원자재형 펀드의 특징은?", "target": "원자재형 펀드의 특징과 장단점, 원자재형 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 혼합형 펀드의 특징은?", "target": "혼합형 펀드의 특징과 장단점, 혼합형 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 성장주 펀드는?", "target": "성장주 펀드의 특징과 장단점, 성장주 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 가치주 펀드는?", "target": "가치주 펀드의 특징과 장단점, 가치주 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 고배당주 펀드는?", "target": "고배당주 펀드의 특징과 장단점, 고배당주 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 글로벌 펀드는?", "target": "글로벌 펀드의 특징과 장단점, 글로벌 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 국내 펀드는?", "target": "국내 펀드의 특징과 장단점, 국내 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 특정 산업 펀드는?", "target": "특정 산업 펀드의 특징과 장단점, 특정 산업 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 특정 국가 펀드는?", "target": "특정 국가 펀드의 특징과 장단점, 특정 국가 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 특정 자산 펀드는?", "target": "특정 자산 펀드의 특징과 장단점, 특정 자산 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 고정 수익 펀드는?", "target": "고정 수익 펀드의 특징과 장단점, 고정 수익 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 변동 수익 펀드는?", "target": "변동 수익 펀드의 특징과 장단점, 변동 수익 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 특정 테마 펀드는?", "target": "특정 테마 펀드의 특징과 장단점, 특정 테마 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 특정 전략 펀드는?", "target": "특정 전략 펀드의 특징과 장단점, 특정 전략 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 특정 목표 펀드는?", "target": "특정 목표 펀드의 특징과 장단점, 특정 목표 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 특정 기간 펀드는?", "target": "특정 기간 펀드의 특징과 장단점, 특정 기간 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 특정 리스크 펀드는?", "target": "특정 리스크 펀드의 특징과 장단점, 특정 리스크 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 특정 수익률 펀드는?", "target": "특정 수익률 펀드의 특징과 장단점, 특정 수익률 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."},
    {"input": "펀드의 특정 비용 펀드는?", "target": "특정 비용 펀드의 특징과 장단점, 특정 비용 펀드를 선택할 때 고려해야 할 사항을 설명해 주세요."}
]


# JSON 파일 저장 시 ensure_ascii=False 옵션 사용
with open('train_data.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)

# BART 모델과 토크나이저 로드
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# 데이터셋 로드 및 전처리 함수
def preprocess_function(examples):
    inputs = examples["input"]
    targets = examples["target"]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
        labels["input_ids"] = [(label if label != tokenizer.pad_token_id else -100) for label in labels["input_ids"]]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# JSON 파일에서 데이터셋 로드
dataset = load_dataset("json", data_files={"train": "train_data.json"}, encoding='utf-8')["train"]

# 데이터셋을 train과 eval로 분할
train_size = int(0.9 * len(dataset))
train_dataset = dataset.select(range(train_size))
eval_dataset = dataset.select(range(train_size, len(dataset)))

# 전처리 함수 적용
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# 훈련 인자 설정
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=150,
    predict_with_generate=True,
    logging_steps=100,
)

# 트레이너 설정
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# 모델 훈련
trainer.train()

# 모델을 저장
torch.save(model.state_dict(), "bart_model.pth")

# 모델을 불러오기 위한 함수
def load_model(model_path):
    model = BartForConditionalGeneration.from_pretrained(model_name)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# 모델을 불러옴
model = load_model("bart_model.pth")

# 질문 변환 함수
def transform_question(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
    output_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return output_text

# 사용자 인터페이스
while True:
    user_input = input("Enter your question about funds (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    transformed_question = transform_question(user_input)
    print(f"Transformed Question: {transformed_question}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
500,0.083300,0.191478
1000,0.021400,0.227179
1500,0.007000,0.263508
2000,0.004200,0.297911
2500,0.002400,0.326845
3000,0.001900,0.279904
3500,0.000900,0.268906
4000,0.000900,0.299029
4500,0.000600,0.298230


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

Enter your question about funds (or type 'exit' to quit): 펀드가 뭐야?
Transformed Question: 펀드가 무엇인지 설명해주세요. 예시와 함께 턬자하는 방법과 이를 통해 얻을 수 있다면 알려월.
Enter your question about funds (or type 'exit' to quit): 펀드 투자의 장단점은 무엇인가요?
Transformed Question: 펀드 투자의 장단점을 실제 사례를 들어 설명해 주세요.
Enter your question about funds (or type 'exit' to quit): 펀드 투자를 시작하려면 어떻게 해야 하나요?
Transformed Question: 펀드 투자를 처음 시작하려고 학는데, 필요한 절차와 준비 사항을 알련주세욀.
Enter your question about funds (or type 'exit' to quit): 펀드의 환매 절차는 어떻게 되나요?
Transformed Question: 펀드의 종류와 환매 시 주의해야 할 사항을 자세히 알고 싶어요.
Enter your question about funds (or type 'exit' to quit): 펀드의 투자 목표는 무엇인가요?
Transformed Question: 펀드의 종류에는 어떤 것들이 있고, 각결의 특징과 투자 목적은 무엇인가요?
Enter your question about funds (or type 'exit' to quit): 펀드의 수익률 결정 요인은?
Transformed Question: 펀드의 수익률은 어떤 요인에 의해 결정되는지 구체적으로 설명해 주세요.
Enter your question about funds (or type 'exit' to quit): 펀드의 기본 원리는?
Transformed Question: 펀드가 무엇인지 설명해주세요. 등)의 특징과 종류와 이를 통해 얻을 수 있는 장점을 도구나 전략을
Enter your question ab